In [1]:
import os
from dotenv import load_dotenv
import openai
import csv
import pandas as pd
from openai import OpenAI
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv("IMDB Dataset.csv")

In [5]:
df_small = df.iloc[200:220]

In [6]:
print(df_small)

                                                review sentiment
200  Interesting and short television movie describ...  negative
201  Any film about WWII made during WWII by a Brit...  positive
202  A brilliant horror film. Utterly gruesome and ...  positive
203  I absolutely love this film. Everything about ...  positive
204  No one can say I wasn't warned as I have read ...  negative
205  A new way to enjoy Goldsworthy's work, Rivers ...  positive
206  If you liked William Hickey in "Prizzi's Honor...  negative
207  I have seen most, if not all of the Laurel & H...  negative
208  I just started watching The Show around July. ...  positive
209  This film is well cast, often silly and always...  positive
210  I have just given a 10 for Thieves Highway, I ...  negative
211  This movie was terrible. The plot was terrible...  negative
212  I strongly dislike this show. I mean, like, ba...  negative
213  Normally I don't like series at all. They're a...  positive
214  holy Sh*t this was g

In [7]:
client = OpenAI()
def get_completion_from_messages(messages, model="gpt-3.5-turbo", temperature=0):
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature,
    )

    return response.choices[0].message.content

In [ ]:
for i in range(len(reviews)):
    prompt = f"""
    Your task is to determine the sentiment of a review of a movie.

    Determine the Sentiment of the Revie below, delimited by triple \
    backticks.

    Give your answer a single word, either "positive" or "negative".

    Review: ```{reviews[i]}```
    """

    response = get_completion(prompt)
    print(i, response, "\n")
